In [1]:
from rtergpy.run import defaults, event, etime2name, src2ergs
from obspy import UTCDateTime
Defaults=defaults()
Event=event()
Defaults.src="RASPISHAKE"
Defaults.network="AM"
Defaults.chan="EHZ"

In [2]:
eloc = [18.56,-73.55,10] 
etime= UTCDateTime(2021,8,14,12,29,8) 
Event.ecount='00'
#Event.newData=False   # use already downloaded data
Event.newData=True
Event.eventname=etime2name(etime,ecount=Event.ecount)
Event.origin=[eloc,etime]
Event.focmech=[266, 51, 34] # phi,delta,lmbda

print(Event.eventname)

2021081400


In [3]:
#Event.newData=True
src2ergs(Defaults=Defaults,Event=Event)

Getting waveforms
ERROR:  coudn't move directory to  /Users/anewman/Documents/Projects/EQerg/rterg_events/events/2021/2021081400/00.bak
Error:  Couldn't create directory:  /Users/anewman/Documents/Projects/EQerg/rterg_events/events/2021/2021081400/00
Checking for stations available within range from IRIS
Getting waves from IRIS


100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.22s/it]

AM.REDDE.00.EHZ failed to download. Continuing..
writing  /Users/anewman/Documents/Projects/EQerg/rterg_events/events/2021/2021081400/00/pkls/Params_2021081400.pkl 
 /Users/anewman/Documents/Projects/EQerg/rterg_events/events/2021/2021081400/00/Params_2021081400.csv
writing  /Users/anewman/Documents/Projects/EQerg/rterg_events/events/2021/2021081400/00/pkls/Wavestream-raw_2021081400.pkl
Working directory now:  /Users/anewman/Documents/Projects/EQerg/rterg_events/events/2021/2021081400/00



/Users/anewman/opt/anaconda3/envs/leah_rtergpy/lib/python3.9/site-packages/rtergpy/run.py:164: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trdf=trdf.append(trstat2pd(tr),ignore_index=True)


Calculating Energy growth with time 
Running fband [0.00333333, 2.0] Hz:


...

Running fband [0.5, 2.0] Hz:


100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 15.36it/s]


Length EBB and EHF 360 360
Calculating TACER Values


100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 25.30it/s]


Median Tacer time = 158.0 -/+ 114.0/164.5 s (25/75th percentile)
From Median Tacer: --------------------------
  Mean BB Energy (Estimated)= 4.50e+16 [Me 8.20]
    1/300 - 2 Hz: 3 traces, 4.50e+16 +- 10^0.15 [J]
    1/2 - 2 Hz: 3 traces, 4.77e+14 +- 10^0.17 [J]
  Mean BB Energy (FM corrected) = 9.42e+16 [Me 8.42]
    1/300 - 2 Hz: 3 traces, 9.42e+16 +- 10^0.15 [J]
    1/2 - 2 Hz: 3 traces, 9.98e+14 +- 10^0.17 [J]
From Per-Station Tacer: ---------------------
  Mean BB Energy (Estimated)= 3.36e+16 [Me 8.12]
    1/300 - 2 Hz: 3 traces, 3.36e+16 +- 10^0.24 [J]
    1/2 - 2 Hz: 3 traces, 3.78e+14 +- 10^0.31 [J]
  Mean BB Energy (FM corrected) = 7.04e+16 [Me 8.33]
    1/300 - 2 Hz: 3 traces, 7.04e+16 +- 10^0.24 [J]
    1/2 - 2 Hz: 3 traces, 7.90e+14 +- 10^0.31 [J]
Making figures

writing results



### stop here

below is running pieces of code.  Everything seems to technically work, but we're only getting a very few stations (and only pacific northwest).  I wonder if this is because of lack of useful information in the FDSN inventories for RS stations.

In [ ]:

from rtergpy.waveforms import getwaves,ErgsFromWaves,loadwaves,gmeanCut,tacer,tacerstats
from rtergpy.waveforms import trstat2pd,e2Me,eventdir
from rtergpy.plotting import tacerplot,Edistplot,Ehistogram,Eazplot, stationEmapPygmt, fbandlabels, Efluxplots
import numpy as np
import pandas as pd
import matplotlib as mpl
import os
    
Defaults.src="RASPISHAKE"
Defaults.network="AM"
Defaults.chan="EHZ"


if Event.newData:
    print("Getting waveforms")
    #st,df=getwaves(eloc,etime,pwindow=Def.waveparams[1],rads=Def.stationrange)
    st,df=getwaves(Defaults=Defaults,Event=Event)
else:
    print("Loading locally stored waveforms")
    try:
        st,df=loadwaves(Defaults=Defaults,Event=Event) 
    except:
        print("Couldn't load data for "+eventname+". Attempting to download:")
        st,df=getwaves(Defaults=Defaults,Event=Event)
if len(st) == 0:
    raise Exception("ERROR: No waveforms retreived.") 
  

In [ ]:
STATCHAN0=''
for tr in st:
    STATCHAN=str(tr.stats.network)+str(tr.stats.station)
    if STATCHAN == STATCHAN0:
        st.remove(tr)
    STATCHAN0=STATCHAN

# work in event directory 
edirit,origwd=eventdir(Defaults=Defaults,Event=Event,create=False,cd=True)

# create an array of station data 
trdf=pd.DataFrame()
for tr in st:
    trdf=trdf.append(trstat2pd(tr),ignore_index=True)

In [ ]:
trdf.head()

In [ ]:
print("Calculating Energy growth with time ")
# Calculate energies over all waves
EBB,EHF,Emd=ErgsFromWaves(st,Defaults,Event)   
print("Length EBB and EHF", len(EBB),len(EHF))
# get tacer values and fist derivatives 
print("Calculating TACER Values")
#kern=10  # 1/2 width of gauss @ 1sig
kern=Defaults.smoothkern
EBBSmooth=EBB.rolling(kern,win_type='gaussian',center=True, closed='both').mean(std=kern/2)
dEBBdt=EBB.diff()
dEBBdtSmooth=EBBSmooth.diff()

EHFSmooth=EHF.rolling(kern,win_type='gaussian',center=True, closed='both').mean(std=kern/2)
dEHFdt=EHF.diff()
dEHFdtSmooth=EHFSmooth.diff()

prePtime=Defaults.waveparams[1][0]
tacerBB=tacer(dEBBdtSmooth,prePtime=prePtime)
tacerHF=tacer(dEHFdtSmooth,prePtime=prePtime)
ttimes,meds = tacerstats(tacerHF) 
ttimeHF,ttimeHF25,ttimeHF75=ttimes
print("Median Tacer time = %.1f -/+ %.1f/%.1f s (25/75th percentile)" %(ttimeHF,ttimeHF25,ttimeHF75))

intval=int(ttimeHF-prePtime)  # median tacer time from start of waveforms
ebbmedtac=np.array(EBB.iloc[intval])  # list of EBB values at *median* tacer
ehfmedtac=np.array(EHF.iloc[intval])  # list of EBB values at *median* tacer
# corrected values for focmech
ebbcorrmedtac=np.array(list(ebbmedtac*np.array(Emd.est2corr)))
ehfcorrmedtac=np.array(list(ehfmedtac*np.array(Emd.est2corr)))

# Energy values at per station tacer
ebbpertac=[]
ehfpertac=[]
for i in range(0,len(meds)):
    ebbpertac = np.append(ebbpertac, EBB.iloc[meds['time at max'][i]-prePtime][i])
    ehfpertac = np.append(ehfpertac, EHF.iloc[meds['time at max'][i]-prePtime][i])

ebbcorrpertac=np.array(list(ebbpertac*np.array(Emd.est2corr)))
ehfcorrpertac=np.array(list(ehfpertac*np.array(Emd.est2corr)))

# cutoff=15 # 15x +/-  # moved into defaults
cutoff=Defaults.cutoff
labelbb,labelhf=fbandlabels(Emd)

print("From Median Tacer: --------------------------")
ebbmedtacmean,keepbb=gmeanCut(ebbmedtac,cutoff=cutoff)
ehfmedtacmean,keephf=gmeanCut(ehfmedtac,cutoff=cutoff)
ebbmedtacmeanerr10=np.std(np.log10(keepbb))
ehfmedtacmeanerr10=np.std(np.log10(keephf))
print("  Mean BB Energy (Estimated)= %.2e [Me %.2f]" %(ebbmedtacmean,e2Me(ebbmedtacmean)))
print("    %s: %d traces, %.2e +- 10^%.2f [J]" %(labelbb,len(keepbb), ebbmedtacmean, ebbmedtacmeanerr10))
print("    %s: %d traces, %.2e +- 10^%.2f [J]" %(labelhf,len(keephf), ehfmedtacmean, ehfmedtacmeanerr10))
ebbcorrmedtacmean,keepbb=gmeanCut(ebbcorrmedtac,cutoff=cutoff)
ehfcorrmedtacmean,keephf=gmeanCut(ehfcorrmedtac,cutoff=cutoff)
ebbcorrmedtacmeanerr10=np.std(np.log10(keepbb))
ehfcorrmedtacmeanerr10=np.std(np.log10(keephf))
print("  Mean BB Energy (FM corrected) = %.2e [Me %.2f]" %(ebbcorrmedtacmean,e2Me(ebbcorrmedtacmean)))
print("    %s: %d traces, %.2e +- 10^%.2f [J]" %(labelbb,len(keepbb), ebbcorrmedtacmean, ebbcorrmedtacmeanerr10))
print("    %s: %d traces, %.2e +- 10^%.2f [J]" %(labelhf,len(keephf), ehfcorrmedtacmean, ehfcorrmedtacmeanerr10))

print("From Per-Station Tacer: ---------------------")
ebbpertacmean,keepbb=gmeanCut(ebbpertac,cutoff=cutoff)
ehfpertacmean,keephf=gmeanCut(ehfpertac,cutoff=cutoff)
ebbpertacmeanerr10=np.std(np.log10(keepbb))
ehfpertacmeanerr10=np.std(np.log10(keephf))
print("  Mean BB Energy (Estimated)= %.2e [Me %.2f]" %(ebbpertacmean,e2Me(ebbpertacmean)))
print("    %s: %d traces, %.2e +- 10^%.2f [J]" %(labelbb,len(keepbb), ebbpertacmean, ebbpertacmeanerr10))
print("    %s: %d traces, %.2e +- 10^%.2f [J]" %(labelhf,len(keephf), ehfpertacmean, ehfpertacmeanerr10))
ebbcorrpertacmean,keepbb=gmeanCut(ebbcorrpertac,cutoff=cutoff)
ehfcorrpertacmean,keephf=gmeanCut(ehfcorrpertac,cutoff=cutoff)
ebbcorrpertacmeanerr10=np.std(np.log10(keepbb))
ehfcorrpertacmeanerr10=np.std(np.log10(keephf))
print("  Mean BB Energy (FM corrected) = %.2e [Me %.2f]" %(ebbcorrpertacmean,e2Me(ebbcorrpertacmean)))
print("    %s: %d traces, %.2e +- 10^%.2f [J]" %(labelbb,len(keepbb), ebbcorrpertacmean, ebbcorrpertacmeanerr10))
print("    %s: %d traces, %.2e +- 10^%.2f [J]" %(labelhf,len(keephf), ehfcorrpertacmean, ehfcorrpertacmeanerr10))
#print("Mean BB Energy (FM corrected) = %.2e [Me %.2f]" %(emeanbbcorr,e2Me(emeanbbcorr)))

# Saving information  ################################
# create dataframe with Event based results
results=pd.DataFrame({
    "eventname":eventname,"iteration":Event.iter,   # name/run
    "etime":Event.origin[1],"elat":Event.origin[0][0],"elon":Event.origin[0][1],"edepth":Event.origin[0][2], "focmech":[Event.focmech],   # Event
    "network":Defaults.network, "chan":Defaults.chan, "stationrange":[Defaults.stationrange], "nstats":len(trdf), #stations 
    "fbands":[Defaults.waveparams[0]], "pwindow":[Defaults.waveparams[1]],  # wave params
    "eventdir":df.eventdir, "modtime":UTCDateTime(),      # where and when processed
    "cutoff":Defaults.cutoff, "ttimes":[ttimes],  # Results (and below)
    "ebbmedtacmean":ebbmedtacmean, "STD10(medtac)":ebbmedtacmeanerr10, "Me(medtac)":e2Me(ebbmedtacmean), "ehfmedtacmean":ehfmedtacmean, "STD10(hfmedtac)":ehfmedtacmeanerr10,
    "ebbcorrmedtacmean":ebbcorrmedtacmean, "STD10(corr)":ebbcorrmedtacmeanerr10, "Me(corr)":e2Me(ebbcorrmedtacmean), "ehfcorrmedtacmean":ehfcorrmedtacmean, "STD10(hfcorr)":ehfcorrmedtacmeanerr10,
    "ebbpertacmean":ebbpertacmean, "STD10(per)":ebbpertacmeanerr10, "Me(per)":e2Me(ebbpertacmean), "ehfpertacmean":ehfpertacmean, "STD10(hfper)":ehfpertacmeanerr10,
    "ebbcorrpertacmean":ebbcorrpertacmean, "STD10(percorr)":ebbcorrpertacmeanerr10, "Me(corrper)":e2Me(ebbcorrpertacmean), "ehfcorrpertacmean":ehfcorrpertacmean, "STD10(hfpercorr)":ehfcorrpertacmeanerr10
    }, dtype=object)

# time-series energy results
Etimeseries=pd.concat([EBB,EHF,EBBSmooth,EHFSmooth,dEBBdtSmooth,dEHFdtSmooth,tacerBB,tacerHF],
    keys=["EBB","EHF","EBBSmooth","EHFSmooth","dEBBdtSmooth","dEHFdtSmooth","tacerBB","tacerHF"])
    # individual key can be extracted using (e.g. Energy.loc["EHF"])

# per-station information
ETace=pd.DataFrame({'tacer':meds['time at max'],
    'ebbmedtac':ebbmedtac, 'ehfmedtac':ehfmedtac,
    'ebbcorrmedtac':ebbcorrmedtac, 'ehfcorrmedtac':ehfcorrmedtac,
    'ebbpertac':ebbpertac, 'ehfpertac':ehfpertac,
    'ebbcorrpertac':ebbcorrpertac, 'ehfcorrpertac':ehfcorrpertac
    })
ETace=ETace.reset_index(drop=True)
StationTacer=pd.concat([trdf,Emd[["estFgP2","FgP2","est2corr"]],ETace],axis=1)

Event.Me=e2Me(ebbpertacmean)
Event.ttime=ttimeHF

print("Making figures\n")
if not os.path.exists('figs'):   # create and go into pkls dir
    os.mkdir('figs')
os.chdir('figs')
showPlots=False
try:
    if not showPlots:
        mpl.use('Agg')  # needed to plot without using the X-session
    # individual plot runs    
    droppcts=[0.5,.25,0.1]
    droptimes=Efluxplots(dEHFdtSmooth, trdf, Event=Event, Defaults=Defaults, pcts=droppcts, show=showPlots)    
    tacerplot(tacerHF,trdf,ttimes,meds,eventname,show=showPlots)
    Edistplot(EBB,EHF,Emd,trdf,eventname,ttimeHF, prePtime=prePtime,show=showPlots,cutoff=cutoff)
    Eazplot(EBB,EHF,Emd,trdf,eventname,ttimeHF, prePtime=prePtime,show=showPlots,cutoff=cutoff)
    Ehistogram(EBB,EHF,Emd,eventname,ttimeHF, prePtime=prePtime,show=showPlots,cutoff=cutoff)
    stationEmapPygmt(EBB,Event.origin[0],trdf,eventname,ttimeHF, prePtime=prePtime,cutoff=15,itername=Event.iter,show=showPlots)
    mpl.pyplot.close('all')  # they don't close themselves
except:
    print("ERROR: Plotting Results for "+eventname) # test 

os.chdir('..')

results["Droptimes"]=[droptimes]

# save results to files
try:
    print("writing results\n")
    # csv    
    results.to_csv("Results_"+eventname+".csv")
    Etimeseries.to_csv("Etimeseries_"+eventname+".csv")
    StationTacer.to_csv("ETacer_"+eventname+".csv") 
    # pkls 
    if not os.path.exists('pkls'):   # create and go into pkls dir
        os.mkdir('pkls')
    os.chdir('pkls')
    results.to_pickle("Results_"+eventname+".pkl")
    Etimeseries.to_pickle("Etimeseries_"+eventname+".pkl")
    StationTacer.to_pickle("Etacer_"+eventname+".pkl")
    os.chdir('..')
except:
    print("ERROR: writing results for"+eventname)


os.chdir(origwd)  # go back to old directory

In [ ]:
showPlots=True
if not showPlots:
    mpl.use('Agg')  # needed to plot without using the X-session
# individual plot runs    
droppcts=[0.5,.25,0.1]
droptimes=Efluxplots(dEHFdtSmooth, trdf, Event=Event, Defaults=Defaults, pcts=droppcts, show=showPlots)    
tacerplot(tacerHF,trdf,ttimes,meds,eventname,show=showPlots)
Edistplot(EBB,EHF,Emd,trdf,eventname,ttimeHF, prePtime=prePtime,show=showPlots,cutoff=cutoff)
Eazplot(EBB,EHF,Emd,trdf,eventname,ttimeHF, prePtime=prePtime,show=showPlots,cutoff=cutoff)
Ehistogram(EBB,EHF,Emd,eventname,ttimeHF, prePtime=prePtime,show=showPlots,cutoff=cutoff)
stationEmapPygmt(EBB,Event.origin[0],trdf,eventname,ttimeHF, prePtime=prePtime,cutoff=15,itername=Event.iter,show=showPlots)
mpl.pyplot.close('all')  # they don't close themselves
  

In [ ]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
rs = Client('RASPISHAKE')

start = UTCDateTime(2022, 6, 7, 0, 0, 0)
end = UTCDateTime(2022, 6, 7, 0, 2, 0)

stn = 'R991C'            # Zhigang's
inv = rs.get_stations(network='AM', station=stn, level='RESP')

stream = rs.get_waveforms('AM', stn, '00', 'EHZ', start, end)
stream.attach_response(inv)

resp_removed = stream.remove_response()

In [ ]:
stream[0].plot();

In [ ]:
print(stream[0])
